<a href="https://colab.research.google.com/github/JudeTulel/KSA/blob/main/Marirtime_survaillance_on_YOLOV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combatting illegal maritime fishing on the Kenyan Coast
Following the call for applications by the Kenya Space agency to develop Space-Bourne solutions to address the challenges facing the Kenya, we heeded the call and developed a solution to combat illegal maritime fishing on the Kenyan Coast. The solution is a YOLO v5 model trained on a comprehensive dataset of maritime sattelite images comprising of a large-scale fine-grainted dataset of 3,435 images from various sensors, satellite platforms, locations, and seasons.

# The Dataset
For any AI or ML model data plays a paramount role. When speaking of data we talk about its quantity and quality. We chose to make use of the ShipRSImageNet datasetwhich was first presented by Z. Zhang, L. Zhang, Y. Wang, P. Feng and R. He in the paper "ShipRSImageNet: A Large-Scale Fine-Grained Dataset for Ship Detection in High-Resolution Optical Remote Sensing Images," in IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, vol. 14, pp. 8458-8472, 2021, doi: 10.1109/JSTARS.2021.3104230.

Each image is around 930×930 pixels and contains ships with different scales, orientations, and aspect ratios. The images are annotated by experts in satellite image interpretation, categorized into 50 object categories images. The fully annotated ShipRSImageNet contains 17,573 ship instances.

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [7]:
!git clone https://github.com/ultralytics/yolov5.git  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.1+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/107.7 GB disk)


## Downloading the dataset

In [8]:
# download the zip file from google drive

!gdown --id '1wApkaSoa9mXRfXQiq6lTtlVrv4cSc6vv' # replace 'your_google_drive_file_id' with actual file id



/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1wApkaSoa9mXRfXQiq6lTtlVrv4cSc6vv
From (redirected): https://drive.google.com/uc?id=1wApkaSoa9mXRfXQiq6lTtlVrv4cSc6vv&confirm=t&uuid=f624bbee-5913-4faa-ab13-f2e956790d65
To: /content/yolov5/yolov5/ShipRSImageNet_V1.zip
100% 4.58G/4.58G [01:11<00:00, 64.4MB/s]


In [9]:
# unzipping the dataset
!unzip ShipRSImageNet_V1.zip

Streaming output truncated to the last 5000 lines.
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100001367.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100001429.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/2545__2760_920.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/000281.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100000723.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100000910.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/000518.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/000524.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/002333.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/001148.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100001398.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100000938.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotations/100001171.xml  
  inflating: ShipRSImageNet_V1/VOC_Format/Annotat

In [13]:
# prompt:  script to move images files from /content/yolov5/yolov5/ShipRSImageNet_V1/VOC_Format/JPEGImages/ to /content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images/

import os
import shutil

# Define source and destination directories
source_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/VOC_Format/JPEGImages/'
destination_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images/'

# Iterate over files in source directory
for filename in os.listdir(source_dir):
  # Construct full file paths
  source_path = os.path.join(source_dir, filename)
  destination_path = os.path.join(destination_dir, filename)
  # Move file
  shutil.move(source_path, destination_path)


## Converting COCO JSON to YOLOtxt format


In [14]:
import json
import os
from pathlib import Path

def coco_to_yolo(coco_json, images_dir, labels_dir):
    # Load COCO JSON
    with open(coco_json, 'r') as f:
        coco_data = json.load(f)

    # Create directories if they don't exist
    Path(labels_dir).mkdir(parents=True, exist_ok=True)

    # Mapping of category ID to category name
    category_map = {cat['id']: cat['name'] for cat in coco_data['categories']}

    # Process each image
    for image in coco_data['images']:
        image_id = image['id']
        image_filename = image['file_name']
        image_path = os.path.join(images_dir, image_filename)
        txt_filename = os.path.splitext(image_filename)[0] + '.txt'
        txt_path = os.path.join(labels_dir, txt_filename)

        # Open the corresponding .txt file for writing
        with open(txt_path, 'w') as txt_file:
            # Get annotations for this image
            annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

            for ann in annotations:
                # Convert COCO bbox format [x, y, width, height] to YOLO format [x_center, y_center, width, height]
                bbox = ann['bbox']
                x, y, width, height = bbox
                x_center = (x + width / 2) / image['width']
                y_center = (y + height / 2) / image['height']
                width /= image['width']
                height /= image['height']

                # Get the class ID and map it to a class name
                class_id = ann['category_id']
                class_name = category_map[class_id]

                # Write to the .txt file in YOLO format
                txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Parameters
coco_json = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/ShipRSImageNet_bbox_train_level_3.json'  # Path to your COCO JSON file
images_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images'  # Path to your images directory
labels_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/labels/train'  # Path to the directory where YOLO .txt files should be saved

coco_to_yolo(coco_json, images_dir, labels_dir)


In [15]:
#for validation data

val_json = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/ShipRSImageNet_bbox_val_level_3.json'
images_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images'
labels_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/labels/val'
coco_to_yolo(val_json, images_dir, labels_dir)


In [16]:

import os
import shutil

# Define source and destination directories
source_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images'  # Directory containing all images
val_labels_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/labels/val'  # Directory with validation labels
destination_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images/val'  # Directory to move validation images

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get a list of validation image filenames from the labels
val_image_filenames = []
for filename in os.listdir(val_labels_dir):
    if filename.endswith('.txt'):
        val_image_filenames.append(filename.replace('.txt', '.bmp'))

# Move validation images
for filename in val_image_filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    if os.path.exists(source_path):  # Check if the image exists
        shutil.move(source_path, destination_path)


In [17]:
#moving the image for train to train folder
import os
import shutil

# Define source and destination directories
source_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images'  # Directory containing all images
val_labels_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/labels/train'  # Directory with validation labels
destination_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images/train'  # Directory to move validation images

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get a list of validation image filenames from the labels
val_image_filenames = []
for filename in os.listdir(val_labels_dir):
    if filename.endswith('.txt'):
        val_image_filenames.append(filename.replace('.txt', '.bmp'))

# Move validation images
for filename in val_image_filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    if os.path.exists(source_path):  # Check if the image exists
        shutil.move(source_path, destination_path)


In [18]:
# prompt: read the file names in test.txt and move image files with matching names to /content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/labels/test

# Read filenames from test.txt
with open('/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/test.txt', 'r') as f:
    test_filenames = [line.strip() for line in f]

# Define source and destination directories
source_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images'
destination_dir = '/content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/images/test'

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move test images
for filename in test_filenames:
    source_path = os.path.join(source_dir, filename + '.bmp')  # Assuming images are .bmp
    destination_path = os.path.join(destination_dir, filename + '.bmp')
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)


# Training the model
Now the the dataset is completly downloaded an extracted into the yolov5 folder we will procced with training

We chose to train it

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 960 --batch 4 --epochs 3 --data VOC1.yaml --weights yolov5s.pt --cache

2024-08-17 17:03:27.745512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 17:03:27.770351: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 17:03:27.777746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=VOC1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=4, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimize

In [ ]:
import os
img_dir = '/content/yolov5/ShipRSImageNet_V1/VOC_Format/JPEGImages/'
img_files = os.listdir(img_dir)
print(img_files)


['002678.bmp', '004068.bmp', '100000891.bmp', '002585.bmp', '1397__920_0.bmp', '100001187.bmp', '000420.bmp', '002730.bmp', '100001664.bmp', '1907__1840_1979.bmp', '100000839.bmp', '1217__1963_920.bmp', '000627.bmp', '1399__1840_0.bmp', '004415.bmp', '003472.bmp', '000825.bmp', '001753.bmp', '002859.bmp', '100001206.bmp', '100000957.bmp', '2010__2760_1840.bmp', '000587.bmp', '100001168.bmp', '100001087.bmp', '100001014.bmp', '100001290.bmp', '002511.bmp', '100001350.bmp', '2010__920_2250.bmp', '1931__920_0.bmp', '1858__2012_0.bmp', '100001382.bmp', '100000915.bmp', '004361.bmp', '100000770.bmp', '100001666.bmp', '003264.bmp', '001124.bmp', '100000005.bmp', '000651.bmp', '004373.bmp', '004034.bmp', '004500.bmp', '000351.bmp', '100001676.bmp', '100000973.bmp', '000757.bmp', '000835.bmp', '000431.bmp', '1c4f227ca.bmp', '100001225.bmp', '000664.bmp', '001740.bmp', '2560__2760_1840.bmp', '001759.bmp', '004290.bmp', '000445.bmp', '100001175.bmp', '100001004.bmp', '100000737.bmp', '000223.bmp

# Detect

`detect.py` runs YOLOv5 inference on a variety of sources and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

## Local Logging

Training results are automatically logged with [Tensorboard](https://www.tensorflow.org/tensorboard) and [CSV](https://github.com/ultralytics/yolov5/pull/4148) loggers to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc.

This directory contains train and val statistics, mosaics, labels, predictions and augmentated mosaics, as well as metrics and charts including precision-recall (PR) curves and confusion matrices.

<img alt="Local logging results" src="https://user-images.githubusercontent.com/26833433/183222430-e1abd1b7-782c-4cde-b04d-ad52926bf818.jpg" width="1280"/>


# Appendix

Additional content below.